# Publish to GemFire

This notebook defines a magic command that publishes models/scores in GPDB to GemFire to support applications that require real-time responses. 

First, start GemFire REST service by following instructions provided in [this GitHub project](https://github.com/pivotalsoftware/dsmicro).

## Imports

In [4]:
import ConfigParser
import os
import requests

## Construct the base URL of GemFire REST interface

The magic command need to have the access to GemFire REST service. GemFire's ``http-service-bind-address`` (host) and ``http-service-port`` (port) need to be provided in a configuration file (e.g. add the following to ~/.dslab_user.cred) like below: 

```
[gemfire_rest_config]
host: localhost
port: 8080
```

In [14]:
GEMFIRE_CONFIG_FILE = os.path.join(os.path.expanduser('~'), '.dslab_user.cred')

def fetchGemFireBaseUrl(rest_config_file=GEMFIRE_CONFIG_FILE):
    """
       Read GemFire REST interface base URL
    """
    #Read GemFire REST service configuration from user supplied file
    conf = ConfigParser.ConfigParser()
    conf.read(rest_config_file)
    #host, port
    host = conf.get('gemfire_rest_config','host')
    port = conf.get('gemfire_rest_config','port')

    #Construct base URL
    base_url =  "http://" + host + ":" + str(port) + "/gemfire-api/v1/"
    return base_url

## Find the mapped GemFire region

A GPDB table need to be mapped to a GemFire region. The mapping defines which GemFire region a particular GPDB table is going to be loaded to. It need to be provided in a configuration file (e.g. add the following to ~/.dslab_user.cred) like below:

```
[table_region_mapping]
gp_table_name_1: GemFire_region_name_1
gp_table_name_2: GemFire_region_name_2
```
The mapping need to be consistent with the table-to-region mapping specified in the GemFire cache configuration file (cf. server-cache.xml in [this GitHub project](https://github.com/pivotalsoftware/dsmicro)).



In [22]:
GEMFIRE_CONFIG_FILE = os.path.join(os.path.expanduser('~'), '.dslab_user.cred')

def findMappedRegion(table, gemfire_config_file=GEMFIRE_CONFIG_FILE):
    """
       Find a GPDB table's corresponding GemFire region
    """
    #Read the table-to-region mapping from user supplied file
    conf = ConfigParser.ConfigParser()
    conf.read(gemfire_config_file)
    return conf.get('table_region_mapping', str(table))

## Execute a GemFire function on a region

In [12]:
def executeGemfireFunction(function_id, region_name):
    """
       Execute a Gemfire function on a user-specified region
    """
    #Construct function URL
    url = fetchGemfireBaseUrl() + "functions/" + str(function_id)

    #Run function
    r = requests.post(url,params={'onRegion':str(region_name)})
    return r.text

## Define a magic command to publish a GPDB table

In [20]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

@register_cell_magic
def publish(line, cell):
    """
        Publish a GPDB table to a Gemfire region.
    """
    #Find the mapped Gemfire region
    regionName = findMappedRegion(cell);
    #Execute the Gemfire LoadFromGPDB function on a the mapped region
    return executeGemfireFunction('LoadFromGPDB', regionName)      

### Use the `publish` command to publish a GPDB table to its corresponding region in GemFire

In [21]:
%%publish
sentiment_score_test

u'[ "Loaded 10 entities on region SentimentScore" ]'

The magic command publishes the ``sentiment_score_test`` table to GemFire. The command essentially invokes the GemFire server-side function ``LoadFromGPDB()`` on the mapped region through GemFire REST interface.